In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '128 tBodyGyro-mad()-Y',
 '141 tBodyGyro-iqr()-Y',
 '428 fBodyGyro-std()-Y',
 '434 fBodyGyro-max()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Activity_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Activity_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)
    
class Subject_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Subject_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 5)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Activity_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 3.273602604866028, Final Batch Loss: 1.0973317623138428
Epoch 2, Loss: 3.2591891288757324, Final Batch Loss: 1.0855427980422974
Epoch 3, Loss: 3.2668423652648926, Final Batch Loss: 1.1006696224212646
Epoch 4, Loss: 3.2474950551986694, Final Batch Loss: 1.072757363319397
Epoch 5, Loss: 3.2402745485305786, Final Batch Loss: 1.070950984954834
Epoch 6, Loss: 3.2520869970321655, Final Batch Loss: 1.1045938730239868
Epoch 7, Loss: 3.234590172767639, Final Batch Loss: 1.0795608758926392
Epoch 8, Loss: 3.2186944484710693, Final Batch Loss: 1.0694972276687622
Epoch 9, Loss: 3.206766963005066, Final Batch Loss: 1.0504335165023804
Epoch 10, Loss: 3.1965088844299316, Final Batch Loss: 1.0593514442443848
Epoch 11, Loss: 3.1868953704833984, Final Batch Loss: 1.0628448724746704
Epoch 12, Loss: 3.162538766860962, Final Batch Loss: 1.0416103601455688
Epoch 13, Loss: 3.1544206142425537, Final Batch Loss: 1.0482701063156128
Epoch 14, Loss: 3.135264277458191, Final Batch Loss: 1.05851387977

Epoch 116, Loss: 0.1925605647265911, Final Batch Loss: 0.03511438146233559
Epoch 117, Loss: 0.16491127014160156, Final Batch Loss: 0.06668539345264435
Epoch 118, Loss: 0.1934518963098526, Final Batch Loss: 0.058235447853803635
Epoch 119, Loss: 0.18646077439188957, Final Batch Loss: 0.07962494343519211
Epoch 120, Loss: 0.21188142150640488, Final Batch Loss: 0.12904998660087585
Epoch 121, Loss: 0.15938770398497581, Final Batch Loss: 0.032909493893384933
Epoch 122, Loss: 0.18163393065333366, Final Batch Loss: 0.058160945773124695
Epoch 123, Loss: 0.18931641802191734, Final Batch Loss: 0.06396974623203278
Epoch 124, Loss: 0.1383473426103592, Final Batch Loss: 0.040840424597263336
Epoch 125, Loss: 0.20334388129413128, Final Batch Loss: 0.028939275071024895
Epoch 126, Loss: 0.13805591687560081, Final Batch Loss: 0.06108696758747101
Epoch 127, Loss: 0.1559332236647606, Final Batch Loss: 0.046549342572689056
Epoch 128, Loss: 0.22111482545733452, Final Batch Loss: 0.11887584626674652
Epoch 129,

Epoch 233, Loss: 0.11902426555752754, Final Batch Loss: 0.03559228777885437
Epoch 234, Loss: 0.05988907255232334, Final Batch Loss: 0.020376810804009438
Epoch 235, Loss: 0.061312668956816196, Final Batch Loss: 0.009508353658020496
Epoch 236, Loss: 0.05790054425597191, Final Batch Loss: 0.010287338867783546
Epoch 237, Loss: 0.10141701344400644, Final Batch Loss: 0.009012934751808643
Epoch 238, Loss: 0.10098731983453035, Final Batch Loss: 0.04806597903370857
Epoch 239, Loss: 0.07492252299562097, Final Batch Loss: 0.005450645927339792
Epoch 240, Loss: 0.06280092243105173, Final Batch Loss: 0.020789803937077522
Epoch 241, Loss: 0.061296334490180016, Final Batch Loss: 0.015611207112669945
Epoch 242, Loss: 0.08135801879689097, Final Batch Loss: 0.007470607291907072
Epoch 243, Loss: 0.0874966960400343, Final Batch Loss: 0.00944504700601101
Epoch 244, Loss: 0.0977434441447258, Final Batch Loss: 0.01755780167877674
Epoch 245, Loss: 0.07342781499028206, Final Batch Loss: 0.024539636448025703
Epo

Epoch 353, Loss: 0.04172943066805601, Final Batch Loss: 0.011520630680024624
Epoch 354, Loss: 0.07999766850844026, Final Batch Loss: 0.03833454102277756
Epoch 355, Loss: 0.08983867522329092, Final Batch Loss: 0.06580700725317001
Epoch 356, Loss: 0.029732541646808386, Final Batch Loss: 0.006111005786806345
Epoch 357, Loss: 0.05542023712769151, Final Batch Loss: 0.0043483939953148365
Epoch 358, Loss: 0.05631935130804777, Final Batch Loss: 0.02519305609166622
Epoch 359, Loss: 0.060047279112041, Final Batch Loss: 0.03818345069885254
Epoch 360, Loss: 0.1204922879114747, Final Batch Loss: 0.09524167329072952
Epoch 361, Loss: 0.03783165640197694, Final Batch Loss: 0.005361140239983797
Epoch 362, Loss: 0.0637251827865839, Final Batch Loss: 0.01776045560836792
Epoch 363, Loss: 0.02543999534100294, Final Batch Loss: 0.010795324109494686
Epoch 364, Loss: 0.04429658246226609, Final Batch Loss: 0.003415716113522649
Epoch 365, Loss: 0.04839428002014756, Final Batch Loss: 0.0020453506149351597
Epoch 

Epoch 471, Loss: 0.03461610805243254, Final Batch Loss: 0.010352828539907932
Epoch 472, Loss: 0.03314841887913644, Final Batch Loss: 0.011439623311161995
Epoch 473, Loss: 0.0195812996244058, Final Batch Loss: 0.0028583102393895388
Epoch 474, Loss: 0.02770837559364736, Final Batch Loss: 0.002925923326984048
Epoch 475, Loss: 0.014798010932281613, Final Batch Loss: 0.006805047392845154
Epoch 476, Loss: 0.026306360494345427, Final Batch Loss: 0.00976055022329092
Epoch 477, Loss: 0.009546041488647461, Final Batch Loss: 0.004230639897286892
Epoch 478, Loss: 0.03618665714748204, Final Batch Loss: 0.0090542733669281
Epoch 479, Loss: 0.016112295212224126, Final Batch Loss: 0.011698158457875252
Epoch 480, Loss: 0.007614632602781057, Final Batch Loss: 0.001744431909173727
Epoch 481, Loss: 0.017679677112028003, Final Batch Loss: 0.002170545281842351
Epoch 482, Loss: 0.020669336314313114, Final Batch Loss: 0.0012746063293889165
Epoch 483, Loss: 0.022489840164780617, Final Batch Loss: 0.011339335702

...

Epoch 4532, Loss: 0.0002401984020252712, Final Batch Loss: 7.084798562573269e-05
Epoch 4533, Loss: 0.00012393274391797604, Final Batch Loss: 8.053660712903365e-05
Epoch 4534, Loss: 0.002917110308771953, Final Batch Loss: 0.0026414599269628525
Epoch 4535, Loss: 0.018636283046362223, Final Batch Loss: 3.1824325560592115e-05
Epoch 4536, Loss: 0.002677246244275011, Final Batch Loss: 5.6319055147469044e-05
Epoch 4537, Loss: 0.000143221204780275, Final Batch Loss: 4.561085734167136e-05
Epoch 4538, Loss: 0.0003980901346949395, Final Batch Loss: 0.00029753250419162214
Epoch 4539, Loss: 0.00017842737725004554, Final Batch Loss: 9.086622594622895e-05
Epoch 4540, Loss: 0.00030122693215162144, Final Batch Loss: 4.716059720522026e-06
Epoch 4541, Loss: 0.004015049253212055, Final Batch Loss: 0.00014457147335633636
Epoch 4542, Loss: 0.0013342053971427958, Final Batch Loss: 0.0009095488348975778
Epoch 4543, Loss: 0.000763748241297435, Final Batch Loss: 7.137995271477848e-05
Epoch 4544, Loss: 0.0032329

Epoch 4638, Loss: 0.0002535334606363904, Final Batch Loss: 4.3345749872969463e-05
Epoch 4639, Loss: 0.0001609183400432812, Final Batch Loss: 2.1561525500146672e-05
Epoch 4640, Loss: 0.0004198746978545387, Final Batch Loss: 0.00019942614017054439
Epoch 4641, Loss: 9.366564518131781e-05, Final Batch Loss: 4.7716184781165794e-05
Epoch 4642, Loss: 0.0004593930407281732, Final Batch Loss: 1.987115501833614e-05
Epoch 4643, Loss: 0.00024094566106214188, Final Batch Loss: 3.12474076054059e-05
Epoch 4644, Loss: 0.00019509706953613204, Final Batch Loss: 1.5476540283998474e-05
Epoch 4645, Loss: 0.00016053413673944306, Final Batch Loss: 7.288642518687993e-05
Epoch 4646, Loss: 0.0007273242053997819, Final Batch Loss: 7.825225475244224e-05
Epoch 4647, Loss: 0.0007392753004751285, Final Batch Loss: 3.7157878978177905e-05
Epoch 4648, Loss: 0.005924013395997463, Final Batch Loss: 3.109639146714471e-05
Epoch 4649, Loss: 0.00011826313311757986, Final Batch Loss: 3.4440570743754506e-05
Epoch 4650, Loss: 0

Epoch 4747, Loss: 0.00041574636543373344, Final Batch Loss: 1.109710046875989e-05
Epoch 4748, Loss: 0.00047599998652003706, Final Batch Loss: 3.784511500271037e-05
Epoch 4749, Loss: 9.270150280826783e-05, Final Batch Loss: 1.4626773008785676e-05
Epoch 4750, Loss: 0.00013804187892674236, Final Batch Loss: 1.3221969311416615e-05
Epoch 4751, Loss: 0.00020926055913150776, Final Batch Loss: 7.055053720250726e-05
Epoch 4752, Loss: 0.00032655960751526436, Final Batch Loss: 1.1507341923788772e-06
Epoch 4753, Loss: 0.00012084412264812272, Final Batch Loss: 3.312172702862881e-05
Epoch 4754, Loss: 8.087160131253768e-05, Final Batch Loss: 1.3997825590195134e-05
Epoch 4755, Loss: 0.0005408252204688324, Final Batch Loss: 5.208656602917472e-06
Epoch 4756, Loss: 0.0002469463179295417, Final Batch Loss: 0.00021477956033777446
Epoch 4757, Loss: 8.196605631383136e-05, Final Batch Loss: 2.948331712104846e-06
Epoch 4758, Loss: 0.0002708219672058476, Final Batch Loss: 2.3038137442199513e-05
Epoch 4759, Loss

Epoch 4848, Loss: 0.00013462985953083262, Final Batch Loss: 6.219387432793155e-05
Epoch 4849, Loss: 0.011204798458493315, Final Batch Loss: 0.0001787596702342853
Epoch 4850, Loss: 5.9384625501479604e-05, Final Batch Loss: 5.831621820107102e-06
Epoch 4851, Loss: 6.66325709062221e-05, Final Batch Loss: 2.80602534985519e-06
Epoch 4852, Loss: 0.0001048982517204422, Final Batch Loss: 5.30197894477169e-06
Epoch 4853, Loss: 0.0035103036352666095, Final Batch Loss: 0.0001859363983385265
Epoch 4854, Loss: 0.010930599607490876, Final Batch Loss: 9.08635320229223e-06
Epoch 4855, Loss: 8.246825473179342e-05, Final Batch Loss: 5.345251702237874e-05
Epoch 4856, Loss: 7.391153758362634e-05, Final Batch Loss: 2.060449514829088e-05
Epoch 4857, Loss: 0.00010737769753177417, Final Batch Loss: 7.707690201641526e-06
Epoch 4858, Loss: 0.012818944491300499, Final Batch Loss: 0.012614917941391468
Epoch 4859, Loss: 0.0012182376144664886, Final Batch Loss: 4.623832410288742e-06
Epoch 4860, Loss: 5.1414573135843

Epoch 4960, Loss: 0.0025310521050414536, Final Batch Loss: 1.5147616068134084e-05
Epoch 4961, Loss: 0.0011323230482958024, Final Batch Loss: 0.0010007860837504268
Epoch 4962, Loss: 0.0007829868318367517, Final Batch Loss: 1.1591995644266717e-05
Epoch 4963, Loss: 0.00019909186084987596, Final Batch Loss: 2.435763417452108e-05
Epoch 4964, Loss: 0.00011966903684879071, Final Batch Loss: 7.3393399361521e-05
Epoch 4965, Loss: 6.724739967012283e-05, Final Batch Loss: 4.841541522182524e-05
Epoch 4966, Loss: 0.0005188809400351602, Final Batch Loss: 8.179121505236253e-05
Epoch 4967, Loss: 3.546815287336358e-05, Final Batch Loss: 1.847618113970384e-05
Epoch 4968, Loss: 2.8607332524188678e-05, Final Batch Loss: 1.8735376215772703e-05
Epoch 4969, Loss: 0.00011937878002754587, Final Batch Loss: 4.470812200452201e-05
Epoch 4970, Loss: 5.134458388056373e-05, Final Batch Loss: 3.0265588065958582e-05
Epoch 4971, Loss: 4.824957522941986e-05, Final Batch Loss: 1.0993689102178905e-05
Epoch 4972, Loss: 0.0

In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[62  2  0]
 [ 1 40  0]
 [ 0  0 52]]
              precision    recall  f1-score   support

           0    0.98413   0.96875   0.97638        64
           1    0.95238   0.97561   0.96386        41
           2    1.00000   1.00000   1.00000        52

    accuracy                        0.98089       157
   macro avg    0.97884   0.98145   0.98008       157
weighted avg    0.98109   0.98089   0.98093       157



In [11]:
n_samples = 10

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(n_samples * 5)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_10 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(n_samples * 5)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_11 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_12 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_13 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_14 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_15 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(n_samples * 5) + 1

fake_features = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9, fake_features_10, fake_features_11, fake_features_12,
                               fake_features_13, fake_features_14, fake_features_15))
fake_labels = np.concatenate((y_1, y_2, y_3))

fake_features = torch.Tensor(fake_features)

In [12]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[47  3  0]
 [ 3 47  0]
 [ 0  0 50]]
              precision    recall  f1-score   support

         0.0    0.94000   0.94000   0.94000        50
         1.0    0.94000   0.94000   0.94000        50
         2.0    1.00000   1.00000   1.00000        50

    accuracy                        0.96000       150
   macro avg    0.96000   0.96000   0.96000       150
weighted avg    0.96000   0.96000   0.96000       150



# User Classifier

In [13]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [14]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    elif y[k] == 5:
        y[k] = 2
    elif y[k] == 7:
        y[k] = 3
    else:
        y[k] = 4

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Subject_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [16]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.834184408187866, Final Batch Loss: 1.6053729057312012
Epoch 2, Loss: 4.839436054229736, Final Batch Loss: 1.6326180696487427
Epoch 3, Loss: 4.817689538002014, Final Batch Loss: 1.5912237167358398
Epoch 4, Loss: 4.836143374443054, Final Batch Loss: 1.6288800239562988
Epoch 5, Loss: 4.814775824546814, Final Batch Loss: 1.593578815460205
Epoch 6, Loss: 4.809311270713806, Final Batch Loss: 1.5917668342590332
Epoch 7, Loss: 4.815356373786926, Final Batch Loss: 1.6131302118301392
Epoch 8, Loss: 4.8078086376190186, Final Batch Loss: 1.599535346031189
Epoch 9, Loss: 4.80619752407074, Final Batch Loss: 1.5999796390533447
Epoch 10, Loss: 4.8101547956466675, Final Batch Loss: 1.6124038696289062
Epoch 11, Loss: 4.8039703369140625, Final Batch Loss: 1.6111074686050415
Epoch 12, Loss: 4.802978277206421, Final Batch Loss: 1.6167726516723633
Epoch 13, Loss: 4.7988070249557495, Final Batch Loss: 1.608424186706543
Epoch 14, Loss: 4.78652036190033, Final Batch Loss: 1.604487657546997
Epo

Epoch 127, Loss: 2.459258019924164, Final Batch Loss: 0.8427100777626038
Epoch 128, Loss: 2.408574104309082, Final Batch Loss: 0.7349541187286377
Epoch 129, Loss: 2.4088306427001953, Final Batch Loss: 0.8089219331741333
Epoch 130, Loss: 2.463410794734955, Final Batch Loss: 0.8505508303642273
Epoch 131, Loss: 2.375169575214386, Final Batch Loss: 0.751697301864624
Epoch 132, Loss: 2.3996938467025757, Final Batch Loss: 0.7617265582084656
Epoch 133, Loss: 2.4642420411109924, Final Batch Loss: 0.8779067993164062
Epoch 134, Loss: 2.408451795578003, Final Batch Loss: 0.7613816261291504
Epoch 135, Loss: 2.400020480155945, Final Batch Loss: 0.8255825638771057
Epoch 136, Loss: 2.322398364543915, Final Batch Loss: 0.7710760831832886
Epoch 137, Loss: 2.4244235157966614, Final Batch Loss: 0.7490779757499695
Epoch 138, Loss: 2.3357197642326355, Final Batch Loss: 0.8076683282852173
Epoch 139, Loss: 2.4365898966789246, Final Batch Loss: 0.8571483492851257
Epoch 140, Loss: 2.3424432277679443, Final Bat

Epoch 240, Loss: 1.9851248860359192, Final Batch Loss: 0.6056119799613953
Epoch 241, Loss: 1.8924556374549866, Final Batch Loss: 0.599976122379303
Epoch 242, Loss: 1.9373915791511536, Final Batch Loss: 0.5833808183670044
Epoch 243, Loss: 1.9067396521568298, Final Batch Loss: 0.6418732404708862
Epoch 244, Loss: 2.0281132459640503, Final Batch Loss: 0.6786220669746399
Epoch 245, Loss: 1.9237732291221619, Final Batch Loss: 0.5211618542671204
Epoch 246, Loss: 2.069043457508087, Final Batch Loss: 0.7428763508796692
Epoch 247, Loss: 1.9563947319984436, Final Batch Loss: 0.5984055399894714
Epoch 248, Loss: 2.0257202982902527, Final Batch Loss: 0.7485061287879944
Epoch 249, Loss: 1.947800636291504, Final Batch Loss: 0.6853135824203491
Epoch 250, Loss: 1.987294316291809, Final Batch Loss: 0.6791892647743225
Epoch 251, Loss: 1.9689267873764038, Final Batch Loss: 0.7449610829353333
Epoch 252, Loss: 1.9890769720077515, Final Batch Loss: 0.7946107983589172
Epoch 253, Loss: 1.9182765483856201, Final

Epoch 354, Loss: 1.5973060131072998, Final Batch Loss: 0.6247422099113464
Epoch 355, Loss: 1.6408727169036865, Final Batch Loss: 0.5872305035591125
Epoch 356, Loss: 1.5601516664028168, Final Batch Loss: 0.44505199790000916
Epoch 357, Loss: 1.6409980356693268, Final Batch Loss: 0.6524536609649658
Epoch 358, Loss: 1.6892577409744263, Final Batch Loss: 0.5321382880210876
Epoch 359, Loss: 1.585007220506668, Final Batch Loss: 0.5548485517501831
Epoch 360, Loss: 1.612597405910492, Final Batch Loss: 0.5330271124839783
Epoch 361, Loss: 1.4839994013309479, Final Batch Loss: 0.454484760761261
Epoch 362, Loss: 1.5644255578517914, Final Batch Loss: 0.5506051778793335
Epoch 363, Loss: 1.569868952035904, Final Batch Loss: 0.4896050989627838
Epoch 364, Loss: 1.6309658288955688, Final Batch Loss: 0.5208932161331177
Epoch 365, Loss: 1.6908063888549805, Final Batch Loss: 0.6559338569641113
Epoch 366, Loss: 1.6108065843582153, Final Batch Loss: 0.5982579588890076
Epoch 367, Loss: 1.6264131963253021, Fina

Epoch 476, Loss: 1.3854524791240692, Final Batch Loss: 0.45031315088272095
Epoch 477, Loss: 1.4441445469856262, Final Batch Loss: 0.4933953881263733
Epoch 478, Loss: 1.359555870294571, Final Batch Loss: 0.38606950640678406
Epoch 479, Loss: 1.3899399638175964, Final Batch Loss: 0.43685615062713623
Epoch 480, Loss: 1.2784131467342377, Final Batch Loss: 0.3841024935245514
Epoch 481, Loss: 1.3884690403938293, Final Batch Loss: 0.47645801305770874
Epoch 482, Loss: 1.4242711663246155, Final Batch Loss: 0.4957754611968994
Epoch 483, Loss: 1.3487030565738678, Final Batch Loss: 0.46261492371559143
Epoch 484, Loss: 1.2783931195735931, Final Batch Loss: 0.43475577235221863
Epoch 485, Loss: 1.4119891226291656, Final Batch Loss: 0.5268942713737488
Epoch 486, Loss: 1.3051831126213074, Final Batch Loss: 0.44414862990379333
Epoch 487, Loss: 1.2940952777862549, Final Batch Loss: 0.35766857862472534
Epoch 488, Loss: 1.2998786270618439, Final Batch Loss: 0.44869115948677063
Epoch 489, Loss: 1.33575719594

...

Epoch 4461, Loss: 0.5175869315862656, Final Batch Loss: 0.20400576293468475
Epoch 4462, Loss: 0.48916900157928467, Final Batch Loss: 0.19591180980205536
Epoch 4463, Loss: 0.3830753415822983, Final Batch Loss: 0.10857576876878738
Epoch 4464, Loss: 0.3907480686903, Final Batch Loss: 0.12829622626304626
Epoch 4465, Loss: 0.48197396099567413, Final Batch Loss: 0.17222222685813904
Epoch 4466, Loss: 0.4444807320833206, Final Batch Loss: 0.2062644511461258
Epoch 4467, Loss: 0.5025937408208847, Final Batch Loss: 0.15356917679309845
Epoch 4468, Loss: 0.33970747888088226, Final Batch Loss: 0.08574207872152328
Epoch 4469, Loss: 0.3707076385617256, Final Batch Loss: 0.18414358794689178
Epoch 4470, Loss: 0.45024803280830383, Final Batch Loss: 0.1603337824344635
Epoch 4471, Loss: 0.36999621242284775, Final Batch Loss: 0.09142584353685379
Epoch 4472, Loss: 0.447088323533535, Final Batch Loss: 0.09512247890233994
Epoch 4473, Loss: 0.4059614762663841, Final Batch Loss: 0.10668725520372391
Epoch 4474, L

Epoch 4570, Loss: 0.4024861827492714, Final Batch Loss: 0.09212318807840347
Epoch 4571, Loss: 0.41857102513313293, Final Batch Loss: 0.1206682026386261
Epoch 4572, Loss: 0.4701644256711006, Final Batch Loss: 0.21187938749790192
Epoch 4573, Loss: 0.44288238883018494, Final Batch Loss: 0.11716386675834656
Epoch 4574, Loss: 0.4317377880215645, Final Batch Loss: 0.12171999365091324
Epoch 4575, Loss: 0.4335754066705704, Final Batch Loss: 0.09323610365390778
Epoch 4576, Loss: 0.4734005033969879, Final Batch Loss: 0.14856591820716858
Epoch 4577, Loss: 0.48505574464797974, Final Batch Loss: 0.22961847484111786
Epoch 4578, Loss: 0.41743429005146027, Final Batch Loss: 0.14707739651203156
Epoch 4579, Loss: 0.40979764610528946, Final Batch Loss: 0.11605644226074219
Epoch 4580, Loss: 0.42518531531095505, Final Batch Loss: 0.16815723478794098
Epoch 4581, Loss: 0.5007206425070763, Final Batch Loss: 0.22993361949920654
Epoch 4582, Loss: 0.5386014729738235, Final Batch Loss: 0.15489190816879272
Epoch 4

Epoch 4683, Loss: 0.47561295330524445, Final Batch Loss: 0.14840169250965118
Epoch 4684, Loss: 0.40202050656080246, Final Batch Loss: 0.12505114078521729
Epoch 4685, Loss: 0.43791109323501587, Final Batch Loss: 0.13643796741962433
Epoch 4686, Loss: 0.277619831264019, Final Batch Loss: 0.0647888258099556
Epoch 4687, Loss: 0.44400258362293243, Final Batch Loss: 0.11164264380931854
Epoch 4688, Loss: 0.29399237036705017, Final Batch Loss: 0.09381839632987976
Epoch 4689, Loss: 0.41566290706396103, Final Batch Loss: 0.10531271994113922
Epoch 4690, Loss: 0.39718589931726456, Final Batch Loss: 0.14470864832401276
Epoch 4691, Loss: 0.5647976100444794, Final Batch Loss: 0.26007816195487976
Epoch 4692, Loss: 0.37194035947322845, Final Batch Loss: 0.0814846083521843
Epoch 4693, Loss: 0.4268931895494461, Final Batch Loss: 0.1786159873008728
Epoch 4694, Loss: 0.39203376322984695, Final Batch Loss: 0.10732672363519669
Epoch 4695, Loss: 0.4080299362540245, Final Batch Loss: 0.0935375988483429
Epoch 46

Epoch 4800, Loss: 0.4411597102880478, Final Batch Loss: 0.11819339543581009
Epoch 4801, Loss: 0.4539145976305008, Final Batch Loss: 0.1296539157629013
Epoch 4802, Loss: 0.3793943151831627, Final Batch Loss: 0.08654855936765671
Epoch 4803, Loss: 0.38016945868730545, Final Batch Loss: 0.1753162145614624
Epoch 4804, Loss: 0.3695887625217438, Final Batch Loss: 0.1047154888510704
Epoch 4805, Loss: 0.35786689817905426, Final Batch Loss: 0.11371342092752457
Epoch 4806, Loss: 0.2856547236442566, Final Batch Loss: 0.10280506312847137
Epoch 4807, Loss: 0.4458269402384758, Final Batch Loss: 0.08767689019441605
Epoch 4808, Loss: 0.4701702296733856, Final Batch Loss: 0.12770985066890717
Epoch 4809, Loss: 0.5321658924221992, Final Batch Loss: 0.23621977865695953
Epoch 4810, Loss: 0.47701306641101837, Final Batch Loss: 0.1547253578901291
Epoch 4811, Loss: 0.42499491572380066, Final Batch Loss: 0.14059363305568695
Epoch 4812, Loss: 0.3802464008331299, Final Batch Loss: 0.11840929090976715
Epoch 4813, 

Epoch 4921, Loss: 0.35542063415050507, Final Batch Loss: 0.09030670672655106
Epoch 4922, Loss: 0.5786376595497131, Final Batch Loss: 0.24663984775543213
Epoch 4923, Loss: 0.3579541891813278, Final Batch Loss: 0.10465724021196365
Epoch 4924, Loss: 0.39362896978855133, Final Batch Loss: 0.12810534238815308
Epoch 4925, Loss: 0.33039139956235886, Final Batch Loss: 0.13981077075004578
Epoch 4926, Loss: 0.36929284781217575, Final Batch Loss: 0.07504294067621231
Epoch 4927, Loss: 0.455357626080513, Final Batch Loss: 0.19964297115802765
Epoch 4928, Loss: 0.3313605412840843, Final Batch Loss: 0.10697030276060104
Epoch 4929, Loss: 0.36921604722738266, Final Batch Loss: 0.09644389897584915
Epoch 4930, Loss: 0.34413255378603935, Final Batch Loss: 0.06044693663716316
Epoch 4931, Loss: 0.33847104758024216, Final Batch Loss: 0.09608764946460724
Epoch 4932, Loss: 0.3701123744249344, Final Batch Loss: 0.1406937539577484
Epoch 4933, Loss: 0.4500255584716797, Final Batch Loss: 0.13939042389392853
Epoch 4

In [17]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[29  0  0  0  0]
 [ 0 27  1  1  2]
 [ 0  1 33  0  0]
 [ 0  0  0 31  0]
 [ 2  0  2  0 28]]
              precision    recall  f1-score   support

           0    0.93548   1.00000   0.96667        29
           1    0.96429   0.87097   0.91525        31
           2    0.91667   0.97059   0.94286        34
           3    0.96875   1.00000   0.98413        31
           4    0.93333   0.87500   0.90323        32

    accuracy                        0.94268       157
   macro avg    0.94370   0.94331   0.94243       157
weighted avg    0.94323   0.94268   0.94188       157



In [18]:
fake_labels = [0] * n_samples + [1] * n_samples + [2] * n_samples + [3] * n_samples + [4] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples + [3] * n_samples + [4] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples + [3] * n_samples + [4] * n_samples
fake_labels = np.asarray(fake_labels)

In [19]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(fake_labels, preds.cpu()))
print(metrics.classification_report(fake_labels, preds.cpu(), digits = 5))

[[22  1  2  1  4]
 [ 1 23  0  2  4]
 [ 0  1 24  1  4]
 [ 0  4  5 20  1]
 [ 3  6  5  0 16]]
              precision    recall  f1-score   support

           0    0.84615   0.73333   0.78571        30
           1    0.65714   0.76667   0.70769        30
           2    0.66667   0.80000   0.72727        30
           3    0.83333   0.66667   0.74074        30
           4    0.55172   0.53333   0.54237        30

    accuracy                        0.70000       150
   macro avg    0.71100   0.70000   0.70076       150
weighted avg    0.71100   0.70000   0.70076       150

